In [1]:
import sys
import os
import pandas as pd

In [2]:
!git clone git@github.com:A-Alaa/ICENODE.git --branch v0.2.03 --single-branch

Cloning into 'ICENODE'...
remote: Enumerating objects: 1957, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 1957 (delta 90), reused 148 (delta 49), pack-reused 1766
Receiving objects: 100% (1957/1957), 17.67 MiB | 1.94 MiB/s, done.
Resolving deltas: 100% (1371/1371), done.
Note: switching to '60390ba21b4574912be048c3e2c0a5f89dfa8e31'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



In [3]:
sys.path.append('./ICENODE')

In [5]:
from icenode.train_icenode_struct import ICENODE
from icenode.train_gram import GRAM
from icenode.train_retain import RETAIN

%load_ext autoreload
%autoreload 2

/home/asem/.conda/envs/icenode/lib/python3.8/site-packages/jax/experimental/optimizers.py:28: FutureWarning: jax.experimental.optimizers is deprecated, import jax.example_libraries.optimizers instead
  warnings.warn('jax.experimental.optimizers is deprecated, '


## Define Directories

In [6]:
HOME = os.environ.get('HOME')

# MIMIC-III Dataset Directory
mimic3_dir = f'{HOME}/GP/ehr-data/mimic3-transforms'

# ICE-NODE/M trained on MIMIC-III training partition (70%)
icenode_m3_dir = f'{HOME}/GP/ehr-data/icenode-m3-exp/v0.2.03M3_icenode_struct_M/trial_393'

# # RETAIN trained on MIMIC-III training partition (70%)
# retain_m3_dir = f'{HOME}/GP/ehr-data/icenode-m3-exp/v0.1.23M3_retain_M/frozen_trial_142'

# # GRU=GRAM/M trained on MIMIC-III training partition (70%)
# gru_m3_dir = f'{HOME}/GP/ehr-data/icenode-m3-exp/v0.1.23M3_gram_M/frozen_trial_615'

# # GRAM=GRAM/G trained on MIMIC-III training partition (70%)
# gram_m3_dir = f'{HOME}/GP/ehr-data/icenode-m3-exp/v0.1.23M3_gram_G/frozen_trial_442'

## Patient Interface for each Model

In [9]:
icnode_patient_interface = ICENODE.create_patient_interface(mimic3_dir, 'M3')

## Dataset Partitioning

In [10]:
import random

# seed 42 is used in all our experiments in this work.
rng = random.Random(42)
subjects_id = list(icnode_patient_interface.subjects.keys())
rng.shuffle(subjects_id)

# splits = train:val:test = 0.7:.15:.15
splits = int(.7 * len(subjects_id)), int(.85 * len(subjects_id))

train_ids = subjects_id[:splits[0]]
valid_ids = subjects_id[splits[0]:splits[1]]
test_ids = subjects_id[splits[1]:]

## Load Configs and Trained Params

In [11]:
from icenode.utils import load_config, load_params

icenode_config = load_config(f'{icenode_m3_dir}/config.json')
icenode_params = load_params(f'{icenode_m3_dir}/step0100_params.pickle')

# retain_config = load_config(f'{retain_m3_dir}/config.json')
# retain_params = load_params(f'{retain_m3_dir}/step0100_params.pickle')

# gru_config = load_config(f'{gru_m3_dir}/config.json')
# gru_params = load_params(f'{gru_m3_dir}/step0100_params.pickle')

# gram_config = load_config(f'{gram_m3_dir}/config.json')
# gram_params = load_params(f'{gram_m3_dir}/step0100_params.pickle')

## Create Model Objects

In [12]:
icenode_config

{'emb': {'diag': {'embeddings_dim': 120}, 'kind': 'matrix'},
 'model': {'loss_half_life': 85,
  'ode_dyn': 'gru',
  'ode_init_var': 0.0010657640560088758,
  'ode_timescale': 54.9987172768043,
  'ode_with_bias': False,
  'state_size': 20,
  'tay_reg': 0},
 'training': {'batch_size': 17,
  'epochs': 20,
  'loss_mixing': {'L_dyn': 0, 'L_l1': 0, 'L_l2': 0, 'L_pred': 1},
  'lr': 0.0005447439175383232,
  'optimizer': 'adam'}}

In [14]:
icenode = ICENODE.create_model(icenode_config, icnode_patient_interface, train_ids, None)
# retain = RETAIN.create_model(retain_config, retain_patient_interface, train_ids, None)
# gru = GRAM.create_model(gru_config, gram_patient_interface, train_ids, None)
# gram = GRAM.create_model(gram_config, gram_patient_interface, train_ids, None)

In [15]:
code_partitions = ICENODE.code_partitions(icnode_patient_interface, train_ids) 

In [16]:
from icenode.metrics import evaluation_table
res = icenode.eval(icenode_config['training']['loss_mixing'], icenode_params, test_ids)

In [17]:
eval_df = evaluation_table({'TST': res}, code_partitions)

/home/asem/.conda/envs/icenode/lib/python3.8/site-packages/jax/_src/numpy/lax_numpy.py:5338: UserWarning: 'kind' argument to argsort is ignored; only 'stable' sorts are supported.
  warnings.warn("'kind' argument to argsort is ignored; only 'stable' sorts "


In [18]:
eval_df

(                           TST
 MACRO-AUC             0.941861
 MICRO-AUC             0.944052
 accuracy              0.821363
 admissions_count   1009.000000
 dyn_loss              0.000000
 dyn_loss_per_week     0.000000
 f1-score              0.142657
 fn                    0.001600
 fp                    0.177037
 l1_loss               0.000000
 l2_loss               0.000000
 loss                  0.090853
 nfe_per_week          1.899258
 nfex1000            119.564003
 npv                   0.998020
 pre_ACC-P0            0.156659
 pre_ACC-P1            0.364396
 pre_ACC-P2            0.434837
 pre_ACC-P3            0.703125
 pre_ACC-P4            0.924873
 precision             0.077448
 prediction_loss       0.090853
 recall                0.902782
 specificity           0.820000
 tn                    0.806501
 tp                    0.014862,
 {'TST_prediction_loss': 0.09085261821746826,
  'TST_loss': 0.09085261821746826,
  'TST_l1_loss': 0.0,
  'TST_l2_loss': 0.0,
  'TST_dyn

## Analyse AUC for Each Admission in the Test Partition

In [20]:
icenode_auc_df = icenode.admissions_auc_scores(icenode_params, test_ids)

In [22]:
icenode_auc_df

,SUBJECT_ID,HADM_ID,HADM_IDX,AUC,N_CODES,NFE
0,19842,169734,1,0.965336,9.0,158.0
1,19842,186772,2,0.950686,16.0,26.0
2,19842,180229,3,0.977252,9.0,26.0
3,19842,122439,4,0.976190,13.0,44.0
4,19842,163738,5,0.990304,13.0,38.0
...,...,...,...,...,...,...
1004,26421,192868,2,0.949838,6.0,92.0
1005,2558,105758,1,0.882936,11.0,44.0
1006,2558,194247,2,0.956189,8.0,26.0
1007,10832,196835,1,0.976478,9.0,206.0


In [16]:
# retain_auc_df = retain.admissions_auc_scores(retain_params, test_ids)

In [17]:
# gru_auc_df = gru.admissions_auc_scores(gru_params, test_ids)

In [18]:
# gram_auc_df = gram.admissions_auc_scores(gram_params, test_ids)

In [29]:
# gram_auc_df.columns

Index(['SUBJECT_ID', 'HADM_ID', 'HADM_IDX', 'AUC', 'N_CODES'], dtype='object')

In [21]:
icenode_auc_df.columns = ['SUBJECT_ID', 'HADM_ID', 'HADM_IDX', 'ICENODE_AUC', 'N_CODES', 'DAYS_AHEAD', 'INTERVAL', 'ICENODE_NFE']

# retain_auc_df.columns = ['SUBJECT_ID', 'HADM_ID', 'HADM_IDX', 'RETAIN_AUC', 'N_CODES']

# gru_auc_df.columns = ['SUBJECT_ID', 'HADM_ID', 'HADM_IDX', 'GRU_AUC', 'N_CODES']

# gram_auc_df.columns = ['SUBJECT_ID', 'HADM_ID', 'HADM_IDX', 'GRAM_AUC', 'N_CODES']


ValueError: Length mismatch: Expected axis has 6 elements, new values have 8 elements

In [ ]:
icenode_auc_df.HADM_ID.nunique()

In [33]:
icenode_auc_df

,SUBJECT_ID,HADM_ID,HADM_IDX,ICENODE_AUC,N_CODES,DAYS_AHEAD,INTERVAL,ICENODE_NFE
0,24198,135189,1,0.976323,9.0,1,1,8.0
1,24198,135189,2,0.976787,9.0,2,1,8.0
2,24198,135189,3,0.977097,9.0,3,1,8.0
3,24198,135189,4,0.976787,9.0,4,1,8.0
4,24198,135189,5,0.976942,9.0,5,1,8.0
...,...,...,...,...,...,...,...,...
9346,11234,150220,2,0.964360,8.0,1267,1266,98.0
9347,11234,150220,3,0.996697,8.0,1268,1,8.0
9348,11234,150220,4,0.998261,8.0,1269,1,8.0
9349,11234,150220,5,0.998435,8.0,1270,1,8.0


In [35]:
icenode_auc_df2 = icenode_auc_df.drop_duplicates(['SUBJECT_ID', 'HADM_ID'])

In [37]:
icenode_auc_df2.ICENODE_AUC.mean()

0.9572364170139803

In [38]:
icenode_auc_df2

,SUBJECT_ID,HADM_ID,HADM_IDX,ICENODE_AUC,N_CODES,DAYS_AHEAD,INTERVAL,ICENODE_NFE
0,24198,135189,1,0.976323,9.0,1,1,8.0
13,24198,157166,14,0.971526,9.0,217,204,32.0
20,24198,116391,21,0.978373,7.0,336,113,26.0
25,97441,138440,1,0.977877,21.0,1,1,8.0
32,97441,194679,8,0.943302,16.0,39,32,20.0
...,...,...,...,...,...,...,...,...
9326,59496,158272,10,0.964569,12.0,20,11,14.0
9333,1949,122008,1,0.976765,11.0,1,1,8.0
9337,1949,196537,5,0.863499,13.0,10,6,14.0
9345,11234,133876,1,0.997925,4.0,1,1,8.0
